In [1]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2
!pip install transformers

Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (3.1.4).
0 upgraded, 0 newly installed, 0 to remove and 114 not upgraded.
Git LFS initialized.
Cloning into 'kogpt2'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 52 (delta 20), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (52/52), 1.52 MiB | 1.49 MiB/s, done.
Filtering content: 100% (2/2), 959.93 MiB | 111.46 MiB/s, done.


In [2]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("kogpt2/vocab.json", "kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
model = GPT2LMHeadModel(config)

model_dir = 'kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [4]:
!git clone https://github.com/songys/Chatbot_data.git
import pandas as pd
data = pd.read_csv('Chatbot_data/ChatbotData.csv')
data.head(10)

fatal: destination path 'Chatbot_data' already exists and is not an empty directory.


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [5]:
added_special_token_num = tokenizer.add_special_tokens(['<s>', '</s>'])
pad_id = tokenizer.token_to_id("<pad>")
tokenizer.enable_padding(pad_id=pad_id, pad_token="<pad>")
tokenizer.enable_truncation(max_length=128)

In [6]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, file_path):
        self.data = []
        self.file_path = file_path
        self.tokenizer = tokenizer
    
    def load_data(self):
        raw_data = pd.read_csv(self.file_path)
        train_data = '<s>'+raw_data['Q']+'</s>'+'<s>'+raw_data['A']+'</s>'
        #<s>안녕하세요</s><s> -> 네, 안녕하세요</s>
        tokenized_train_data = tokenizer.encode_batch(train_data)
        for single_data in tokenized_train_data:
            self.data.append(torch.tensor(single_data.ids).unsqueeze(0))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data[index]
        return item

In [7]:
train_dataset = ChatDataset(tokenizer=tokenizer, file_path='Chatbot_data/ChatbotData.csv')
train_dataset.load_data()

In [8]:
from torch.utils.data import DataLoader

data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [9]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-4, correct_bias=True)

epochs = 3

avg_loss = (0.0, 0.0)
for epoch in range(epochs):
    count=0
    for data in data_loader:
        optimizer.zero_grad()
        data = data.transpose(1,0)
        data = data.to('cuda')
        model = model.to('cuda')
        
        outputs = model(data, labels=data)
        loss, logits = outputs[:2]
        loss = loss.to('cuda')
        loss.backward()
        avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
        optimizer.step()
        if count % 200 == 0:
            print('epoch no.{0}  train ({1}/{2})  loss = {3:.5f}  avg_loss = {4:.5f}' . format(epoch, count, len(data_loader), loss, avg_loss[0] / avg_loss[1]))
        count += 1

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


epoch no.0  train (0/2956)  loss = 2.98889  avg_loss = 2.98889
epoch no.0  train (200/2956)  loss = 0.82482  avg_loss = 1.45899
epoch no.0  train (400/2956)  loss = 1.39784  avg_loss = 1.26724
epoch no.0  train (600/2956)  loss = 1.09379  avg_loss = 1.25043
epoch no.0  train (800/2956)  loss = 0.98521  avg_loss = 1.23152
epoch no.0  train (1000/2956)  loss = 0.89566  avg_loss = 1.18688
epoch no.0  train (1200/2956)  loss = 1.14917  avg_loss = 1.18476
epoch no.0  train (1400/2956)  loss = 1.46387  avg_loss = 1.17913
epoch no.0  train (1600/2956)  loss = 1.10456  avg_loss = 1.14842
epoch no.0  train (1800/2956)  loss = 1.14077  avg_loss = 1.15080
epoch no.0  train (2000/2956)  loss = 1.29914  avg_loss = 1.16103
epoch no.0  train (2200/2956)  loss = 1.37242  avg_loss = 1.13432
epoch no.0  train (2400/2956)  loss = 1.06511  avg_loss = 1.16063
epoch no.0  train (2600/2956)  loss = 1.02764  avg_loss = 1.11990
epoch no.0  train (2800/2956)  loss = 0.89541  avg_loss = 1.08964
epoch no.1  train

In [10]:
torch.save(model.state_dict(), 'my_model.bin')